In [1]:
from pathlib import Path
import os

yr = 2023
d = 20

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = {}
  for l in inp.splitlines():
    if l.startswith('%'):
      t = 'flip-flop'
    elif l.startswith('&'):
      t = 'conjunction'
    elif l.startswith('broadcaster'):
      t = 'broadcaster'
    name = l.split('-')[0].replace('%','').replace('&', '').strip()
    dests = [x.strip() for x in l.split('>')[1].split(',')]
    formatted_input[name] = {'type': t, 'dests': dests}
  return formatted_input

In [3]:
class Pulse_Module:

  def __init__(self, name):
    self.name = name
    self.queue = []
    self.dests = None
    self.sent_low_pulse_timesteps = []
    self.sent_high_pulse_timesteps = []
    self.debug = False
    self.input = []

  def send_pulse(self, ptype, timestep, button_timestep=None):
    if ptype not in ('l', 'h'):
      raise Exception('Invalid Pulse Type')
    for d in self.dests:
      d.queue.insert(0, (self.name, ptype, timestep))
      if ptype == 'l':
        self.sent_low_pulse_timesteps.append(button_timestep)
      else:
        self.sent_high_pulse_timesteps.append(button_timestep)
      if self.debug:
          print(self.name + ' -{}'.format('high' if ptype=='h' else 'low') + ' -> ' + d.name)

  def run(self):
    raise NotImplementedError('No run() Method Defined For ', self.name)

  def add_queue_to_input(self):
    for q in self.queue:
      if q[0] not in self.input:
        self.input.append(q[0])


class Button(Pulse_Module):

  def __init__(self):
    super().__init__('button')

  def press(self, timestep, button_timestep):
    if len(self.dests)!=1 or self.dests[0].name != 'broadcaster':
      raise Exception('Button sending to non-broadcast module')
    self.send_pulse('l', timestep, button_timestep)

  def run(self, timestep):
    pass

class Output(Pulse_Module):

  def send_pulse(self):
    pass

  def run(self):
    for q in self.queue:
      if q[0] not in self.input:
        self.input.append(q[0])
    self.queue = []



class Broadcaster(Pulse_Module):

  def run(self, timestep, button_timestep=None):
    for q in self.queue:
      if q[0] not in self.input:
        self.input.append(q[0])
    if len(self.queue) != 0:
      self.queue.pop()
      self.send_pulse('l', timestep, button_timestep)



class FlipFlop(Pulse_Module):

  def __init__(self, name):
    super().__init__(name)
    self.on = False

  def run(self, timestep, button_timestep=None):
    for q in self.queue:
      if q[0] not in self.input:
        self.input.append(q[0])
    q = [x for x in self.queue if x[2]!=timestep]
    if len(q) != 0:
      pulse = self.queue.pop()
      if pulse[1] == 'l':
        send = 'l' if self.on else 'h'
        self.send_pulse(send, timestep, button_timestep)
        self.on = not self.on


class Conjunction(Pulse_Module):

  def __init__(self, name):
    super().__init__(name)
    self.input_mem = {}


  def run(self, timestep, button_timestep=None):
    for q in self.queue:
      if q[0] not in self.input_mem:
        self.input_mem[q[0]] = q[1]
      if q[0] not in self.input:
        self.input.append(q[0])
          
    q = [x for x in self.queue if x[2]!=timestep]
    if len(q) != 0:
      pulse = self.queue.pop()
      self.input_mem[pulse[0]] = pulse[1]
      if all([x=='h' for x in self.input_mem.values()]):
        self.send_pulse('l', timestep, button_timestep)
      else:
        self.send_pulse('h', timestep, button_timestep)



In [4]:
def build_network(formatted_input):
  # Build the Modules

  for n in formatted_input:
    t = formatted_input[n]['type']
    M = {'broadcaster': Broadcaster,
        'flip-flop': FlipFlop,
        'conjunction': Conjunction
        }[t]

    formatted_input[n]['module'] = M(n)
    if t == 'broadcaster':
      # Add the button to the network
      button_dict = {}
      button_dict['module'] = Button()
      button_dict['module'].dests = [formatted_input[n]['module']]
      button_dict['type'] = ['button']

  assert(len(button_dict) != 0)
  formatted_input['button'] = button_dict
  # Set their destinations and do any other necessary initialization
  for n in formatted_input:
    if n != 'button':
      formatted_input[n]['module'].dests = [formatted_input[d]['module']
                                            if d in formatted_input else Output(d)
                                            for d in formatted_input[n]['dests']]

  # Have every module send a low pulse as a 'ping' so all modules know their
  # inputs (this is particularly important for the Conjunction modules)
  for n in formatted_input:
    formatted_input[n]['module'].send_pulse('l', -1)

  for n in formatted_input:
    formatted_input[n]['module'].add_queue_to_input()

  # Clear queues and set pings back to 0
  for n in formatted_input:
    if isinstance(formatted_input[n]['module'], FlipFlop):
      formatted_input[n]['module'].on = False
    if isinstance(formatted_input[n]['module'], Conjunction):
      formatted_input[n]['module'].input_mem = {k[0]:'l' for k in
                                                formatted_input[n]['module'].queue}
    formatted_input[n]['module'].queue = []
    formatted_input[n]['module'].sent_low_pulse_timesteps = []
    formatted_input[n]['module'].sent_high_pulse_timesteps = []

  return formatted_input

In [5]:
def press_button(network, timestep, button=None):
  import copy
    
  if button is None:
    button = [network[m]['module'] for m in network.keys()
              if network[m]['module'].name=='button'][0]

  
  button_timestep = copy.deepcopy(timestep)
  button.press(timestep, button_timestep)
  while any([len(network[m]['module'].queue)!=0 if m not in ['button'] else False for m in network.keys()]):
    for m in network.keys():
      if m not in ['button']:
        network[m]['module'].run(timestep, button_timestep)
    timestep += 1
  return None



In [6]:
def spam_button(formatted_input, button_presses=1000):
  import copy

  network = build_network(copy.deepcopy(formatted_input))


  for i, b in enumerate(range(button_presses)):
    press_button(network, timestep=i+1)

  return network


def get_pulses_product(formatted_input, button_presses=1000):
  import numpy as np

  network = spam_button(formatted_input, button_presses)

  s_low = 0
  s_high = 0
  for m in set(network.keys()):
    s_low += len(network[m]['module'].sent_low_pulse_timesteps)
    s_high += len(network[m]['module'].sent_high_pulse_timesteps)

  return np.prod(np.array([s_low, s_high], dtype=np.ulonglong))

In [7]:
def find_first_rx_high(formatted_input, initial_button_presses=10000, debug=False):
  import copy
  from math import lcm


  
  rx_inps = [x for x in formatted_input if 'rx'
            in formatted_input[x]['dests']]
  # We make sure there is only one input to 'rx'
  assert(len(rx_inps)==1)
  rx_inp = rx_inps[0]



  network = None
  button_presses = initial_button_presses
  while (network is None
  or not all([len(network[i]['module'].sent_high_pulse_timesteps)!=0
              for i in network[rx_inp]['module'].input])):
    network = spam_button(copy.deepcopy(formatted_input), button_presses)


  # Make sure the one input to 'rx' is a conjunction
  assert(isinstance(network[rx_inp]['module'], Conjunction))
  # This will only work if we all the modules that are inputs to rx's only input are of type Conjunction
  assert(all([isinstance(network[n]['module'], Conjunction) for n in network[rx_inp]['module'].input]))

  # We are trying to find the first instance that each input to the input module to 'rx' sent a high pulse
  # we calculate the lcm of the timesteps of each of the second order input's first high pulse to
  # determine at which timestep they will all send a high pulse to rx's input 
  # (thus rx's input will send a high pulse since it is a conjunction)
  return lcm(*[network[i]['module'].sent_high_pulse_timesteps[0] for
              i in network[rx_inp]['module'].input])

In [8]:
import time

t = time.time()

formatted_input = format_input(inp)

print(get_pulses_product(formatted_input, 1000))
print(find_first_rx_high(formatted_input, initial_button_presses=10000))

print("\nRUNTIME: ", time.time()-t)

743871576
244151741342687

RUNTIME:  4.811162710189819
